In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc

In [8]:
# Replace 'your_server' and 'your_database' with your actual server and database names.
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=mrwhite.intrans.iastate.edu;'
                      'Database=sim_Andalib;'
                      'Trusted_Connection=yes;')
cursor = conn.cursor()
print("Connected to SQL Server successfully!")

Connected to SQL Server successfully!


In [9]:
signal_sql_test = "select * from dbo.signalchanges where studyID = 1150"
signal_sql_df = pd.read_sql(signal_sql_test, conn)
signal_sql_df.head()

C:\Users\ashams\AppData\Local\Temp\ipykernel_27464\2469571255.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  signal_sql_df = pd.read_sql(signal_sql_test, conn)


,StudyID,RunID,Iteration,InsertionDate,SimTime,SC,SG,State
0,1150,1,1,2025-03-16 11:57:00,0.9,4434,2,-1
1,1150,1,1,2025-03-16 11:57:00,0.9,4434,3,-1
2,1150,1,1,2025-03-16 11:57:00,0.9,4434,4,-1
3,1150,1,1,2025-03-16 11:57:00,0.9,4434,5,-1
4,1150,1,1,2025-03-16 11:57:00,0.9,4434,6,-1


In [16]:
bog = np.array(signal_sql_df.loc[(signal_sql_df.SG == 2) & (signal_sql_df.State == 1) & (signal_sql_df.SC == 4433), 'SimTime'])
eog = np.array(signal_sql_df.loc[(signal_sql_df.SG == 2) & (signal_sql_df.State == 0) & (signal_sql_df.SC == 4433), 'SimTime']) 

bog = np.sort(bog)
eog = np.sort(eog)

In [17]:
bog

array([ 10.9,  97. , 220.4, 320.4, 420.4, 520.4, 616.8, 720.4])

In [18]:
eog

array([4.000e-01, 1.400e+00, 2.400e+00, 3.400e+00, 4.400e+00, 4.520e+01,
       1.510e+02, 2.610e+02, 3.610e+02, 4.610e+02, 5.610e+02, 6.610e+02,
       7.536e+02])

In [19]:
while(bog[0] > eog[0]): eog = np.delete(eog, 0)
while(bog[-1] > eog[-1]): bog = np.delete(bog, -1)

In [29]:
signal_df = pd.DataFrame({'bog':bog, 'eog':eog})
signal_df

,bog,eog
0,10.9,45.2
1,97.0,151.0
2,220.4,261.0
3,320.4,361.0
4,420.4,461.0
5,520.4,561.0
6,616.8,661.0
7,720.4,753.6


In [22]:
tt_sql = "select * from dbo.traveltimes where studyID = 1150"
tt_df = pd.read_sql(tt_sql, conn)
tt_df.head()

C:\Users\ashams\AppData\Local\Temp\ipykernel_27464\602762381.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  tt_df = pd.read_sql(tt_sql, conn)


,StudyId,RunID,Iteration,InsertionDate,Time,No_,Veh,VehType,Trav_,Delay_
0,1150,1,1,2025-03-16 11:57:00,492.989315,622,2111,100,76.336298,50.026977
1,1150,1,1,2025-03-16 11:57:00,492.977809,1332,1123,150,59.795587,38.615559
2,1150,1,1,2025-03-16 11:57:00,492.964439,1532,132,100,72.287178,36.198406
3,1150,1,1,2025-03-16 11:57:00,492.900816,99104,241,100,0.327503,0.000000
4,1150,1,1,2025-03-16 11:57:00,493.036041,332,2279,150,19.860415,12.855360


In [24]:
tt_df2 = tt_df.loc[tt_df['No_'] == 642, ['Time', 'Veh','Trav_', 'Delay_' ]]

In [30]:
tt_df2['start_time'] = tt_df2['Time'] - tt_df2['Trav_']
tt_df2['stopbar_time'] = tt_df2['Time'] - tt_df2['Trav_'] + 18.5
tt_df2.sort_values(by='start_time', inplace=True)

In [31]:
tt_df2

,Time,Veh,Trav_,Delay_,start_time,stopbar_time
14750,228.010418,87,60.099829,31.729486,167.910589,186.410589
7819,328.278087,107,44.833094,14.460879,283.444992,301.944992
7835,328.830757,241,37.874287,6.671149,290.956470,309.456470
3601,345.320180,590,29.466841,0.000000,315.853339,334.353339
3651,348.285358,764,29.516435,0.000000,318.768923,337.268923
...,...,...,...,...,...,...
16719,738.669631,1613,42.385833,11.070507,696.283798,714.783798
16840,740.002775,176,40.886959,10.215578,699.115816,717.615816
16718,738.664901,1693,38.070267,6.654767,700.594633,719.094633
16841,740.097871,1866,37.335754,6.585865,702.762117,721.262117


In [33]:
merged_df = pd.merge_asof(tt_df2, signal_df, left_on='stopbar_time', right_on='bog', direction='backward')

In [34]:
merged_df

,Time,Veh,Trav_,Delay_,start_time,stopbar_time,bog,eog
0,228.010418,87,60.099829,31.729486,167.910589,186.410589,97.0,151.0
1,328.278087,107,44.833094,14.460879,283.444992,301.944992,220.4,261.0
2,328.830757,241,37.874287,6.671149,290.956470,309.456470,220.4,261.0
3,345.320180,590,29.466841,0.000000,315.853339,334.353339,320.4,361.0
4,348.285358,764,29.516435,0.000000,318.768923,337.268923,320.4,361.0
...,...,...,...,...,...,...,...,...
93,738.669631,1613,42.385833,11.070507,696.283798,714.783798,616.8,661.0
94,740.002775,176,40.886959,10.215578,699.115816,717.615816,616.8,661.0
95,738.664901,1693,38.070267,6.654767,700.594633,719.094633,616.8,661.0
96,740.097871,1866,37.335754,6.585865,702.762117,721.262117,720.4,753.6


In [35]:
merged_df['codition_check'] = merged_df.bog < merged_df.stopbar_time

In [37]:
merged_df['aog'] =  merged_df.stopbar_time < merged_df.eog

In [41]:
total_aog = len(merged_df.loc[merged_df.aog == 1])
total_n = len(merged_df)

In [42]:
pog = total_aog / total_n
pog

0.3469387755102041